In [2]:
import os
import json
import requests
from dotenv import load_dotenv
from PIL import Image
from transformers import pipeline

load_dotenv()
hf_token = os.getenv("HF_TOKEN")
os.environ["HF_HOME"] = "/media/pc1/Ubuntu/Extend_Data/hf_models"

/media/pc1/Ubuntu/Extend_Data/anaconda3/envs/ares_thanh/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
image = "/media/pc1/Ubuntu/Extend_Data/em_Thanh/pace_up/images/1---A-20-Year-Old-Woman-from-Sudan-With-Fever--_2022_Clinical-Cases-in-Tropi_1.jpeg"
file_path = "/media/pc1/Ubuntu/Extend_Data/em_Thanh/pace_up/cases_test/1---A-20-Year-Old-Woman-from-Sudan-With-Fever--_2022_Clinical-Cases-in-Tropi.json"

In [4]:
# Load JSON
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Combine all fields except "Extracted Figure"
combined_text = ""
for key, value in data.items():
    if key not in ["Extracted Figure", "Further Reading"]:
        combined_text += f"# {key}:\n{value}\n\n"

# Add Descriptions from "Extracted Figure"
descriptions = []
for figure_group in data.get("Extracted Figure", []):
    for figure in figure_group:
        desc = figure.get("Description", "")
        descriptions.append(desc)

if descriptions:
    combined_text += "# Figure Descriptions:\n"
    for desc in descriptions:
        combined_text += f"{desc}\n"

print(combined_text)

# Intro:
1 A 20-Year-Old Woman from Sudan With Fever, Haemorrhage and Shock DANIEL G. BAUSCH

# Clinical Presentation:
History A 20-year-old housewife presents to a hospital in northern Uganda with a 2-day history of fever, severe asthenia, chest and abdominal pain, nausea, vomiting, diarrhoea and slight non-productive cough. The patient is a Sudanese refugee living in a camp in the region. She denies any contact with sick people. Clinical Findings The patient is prostrate and semiconscious on admission. Vital signs: temperature 39.6°C, (103.3°F) blood pressure 90/60mmHg, pulse 90 bpm, and respiratory rate 24 cycles per minute. Physical examination revealed abdominal tenderness, especially in the right upper quadrant, hepatosplenomegaly and bleeding from the gums. The lungs were clear. No rash or lymphadenopathy was noted.

# Questions:
1. Is the patient’s history and clinical presentation consistent with a haemorrhagic fever (HF) syndrome? 2. What degree of nursing precautions need to

In [5]:
image = Image.open(image)

In [ ]:
pipe = pipeline("image-text-to-text", model="google/medgemma-4b-it", token=hf_token)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.06s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


In [9]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "Describe detail the image"}
        ]
    },
]
result = pipe(messages, max_new_tokens=8192)
print(result[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


[{'role': 'user', 'content': [{'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=CMYK size=376x370 at 0x7F7222AA3370>}, {'type': 'text', 'text': 'Describe detail the image'}]}, {'role': 'assistant', 'content': "The image shows a young child with severe facial injuries. The child's eyes are prominent and appear to be bloodshot and possibly injured, with some visible blood around the eyes. The child's face shows signs of trauma, including what appears to be a cut or laceration on the lip, which is also bleeding. There is a significant amount of blood visible around the child's face. The background is somewhat blurred and shows a colorful, possibly patterned cloth, suggesting the child was likely found in or near a makeshift shelter.\n"}]


In [12]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "You are a clinical expert in Tropical and Infectious diseases. This is an image of patient. Can you describe it in detail to support the diagnosis."}
        ]
    },
]
result = pipe(messages, max_new_tokens=8192)
print(result[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


[{'role': 'user', 'content': [{'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=CMYK size=376x370 at 0x7F7222AA3370>}, {'type': 'text', 'text': 'You are a clinical expert in Tropical and Infectious diseases. This is an image of patient. Can you describe it in detail to support the diagnosis.'}]}, {'role': 'assistant', 'content': "This image depicts a child with severe facial trauma, likely inflicted by violence. Key features that support this are:\n\n*   **Severe Bleeding:** There is significant blood present, especially around the mouth and eyes. This indicates a major injury.\n\n*   **Contusion/Bruising:** The face is likely bruised, based on the discoloration.\n\n*   **Facial Distortion:** The facial features are distorted, which could be due to the extent of the injury and potential displacement of tissues.\n\nWithout further clinical information, it is not possible to provide a definitive diagnosis. However, these findings are consistent with physical abuse 

In [14]:
print(result[0]["generated_text"][-1]['content'])

This image depicts a child with severe facial trauma, likely inflicted by violence. Key features that support this are:

*   **Severe Bleeding:** There is significant blood present, especially around the mouth and eyes. This indicates a major injury.

*   **Contusion/Bruising:** The face is likely bruised, based on the discoloration.

*   **Facial Distortion:** The facial features are distorted, which could be due to the extent of the injury and potential displacement of tissues.

Without further clinical information, it is not possible to provide a definitive diagnosis. However, these findings are consistent with physical abuse or severe trauma to the face. A medical professional would need to assess the child's overall condition, conduct a thorough examination, and gather relevant history to establish a diagnosis.



In [ ]:
The image shows a young child with severe facial injuries. The child's eyes are prominent and appear to be bloodshot and possibly injured, with some visible blood around the eyes. The child's face shows signs of trauma, including what appears to be a cut or laceration on the lip, which is also bleeding. There is a significant amount of blood visible around the child's face. The background is somewhat blurred and shows a colorful, possibly patterned cloth, suggesting the child was likely found in or near a makeshift shelter